In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
X=df.copy()
y=X['Outcome']


In [7]:
X['SkinThickness' + '_was_missing'] = X['SkinThickness']==0

In [8]:
X['Insulin' + '_was_missing'] = X['Insulin']==0

In [9]:
# X.Insulin=np.log(1+X.Insulin)

In [10]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,SkinThickness_was_missing,Insulin_was_missing
0,6,148,72,35,0,33.6,0.627,50,1,False,True
1,1,85,66,29,0,26.6,0.351,31,0,False,True
2,8,183,64,0,0,23.3,0.672,32,1,True,True
3,1,89,66,23,94,28.1,0.167,21,0,False,False
4,0,137,40,35,168,43.1,2.288,33,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,False,False
764,2,122,70,27,0,36.8,0.340,27,0,False,True
765,5,121,72,23,112,26.2,0.245,30,0,False,False
766,1,126,60,0,0,30.1,0.349,47,1,True,True


In [11]:
ymean=X.loc[(X['Outcome'] == 1) & (X['Insulin']!= 0)].Insulin.mean()
print(ymean)
X.loc[(X['Outcome'] == 1) & (X['Insulin']== 0),'Insulin']=ymean

206.84615384615384


In [12]:
nmean=X.loc[(X['Outcome'] == 0) & (X['Insulin']!= 0)].Insulin.mean()
print(nmean)
X.loc[(X['Outcome'] == 0) & (X['Insulin']== 0),'Insulin']=nmean

130.28787878787878


In [13]:
ymean=X.loc[(X['Outcome'] == 1) & (X['SkinThickness']!= 0)].SkinThickness.mean()
print(ymean)
X.loc[(X['Outcome'] == 1) & (X['SkinThickness']== 0),'SkinThickness']=ymean

33.0


In [14]:
nmean=X.loc[(X['Outcome'] == 0) & (X['SkinThickness']!= 0)].SkinThickness.mean()
print(nmean)
X.loc[(X['Outcome'] == 0) & (X['SkinThickness']== 0),'SkinThickness']=nmean

27.235457063711912


In [15]:
X.drop(['Outcome'],axis=1,inplace=True)

In [16]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,SkinThickness_was_missing,Insulin_was_missing
0,6,148,72,35.0,206.846154,33.6,0.627,50,False,True
1,1,85,66,29.0,130.287879,26.6,0.351,31,False,True
2,8,183,64,33.0,206.846154,23.3,0.672,32,True,True
3,1,89,66,23.0,94.000000,28.1,0.167,21,False,False
4,0,137,40,35.0,168.000000,43.1,2.288,33,False,False
...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48.0,180.000000,32.9,0.171,63,False,False
764,2,122,70,27.0,130.287879,36.8,0.340,27,False,True
765,5,121,72,23.0,112.000000,26.2,0.245,30,False,False
766,1,126,60,33.0,206.846154,30.1,0.349,47,True,True


In [17]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Pregnancies                768 non-null    int64  
 1   Glucose                    768 non-null    int64  
 2   BloodPressure              768 non-null    int64  
 3   SkinThickness              768 non-null    float64
 4   Insulin                    768 non-null    float64
 5   BMI                        768 non-null    float64
 6   DiabetesPedigreeFunction   768 non-null    float64
 7   Age                        768 non-null    int64  
 8   SkinThickness_was_missing  768 non-null    bool   
 9   Insulin_was_missing        768 non-null    bool   
dtypes: bool(2), float64(4), int64(4)
memory usage: 49.6 KB


In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, test_size=0.2)

In [19]:
from xgboost import XGBClassifier
model = XGBClassifier()
# my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('model', model)
#                      ])
# my_pipeline.fit(X_train, y_train)
model.fit(X,y)

# # Get predictions
predictions = model.predict(X_valid)

# # Calculate MAE
mae = accuracy_score(y_valid,predictions)
print(mae)

1.0


In [20]:
cols=list(X.columns)
cols

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'SkinThickness_was_missing',
 'Insulin_was_missing']

In [21]:
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
# categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
# print(numerical_cols)
# print(categorical_cols)

In [22]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=0,strategy='mean'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
    ])


In [23]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
xgb1 = XGBClassifier()
parameters = {'learning_rate': [0.03, 0.05,0.1], #so called `eta` value
              'n_estimators': [50,100,250]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 3,
                        scoring='accuracy'
                        )

In [24]:
# xgb_grid.fit(X,y)

In [25]:
cv_pipeline = Pipeline(steps=[('preprocessor',preprocessor),('model', xgb_grid)])
cv_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(missing_values=0))]),
                                                  ['Pregnancies', 'Glucose',
                                                   'BloodPressure',
                                                   'SkinThickness', 'Insulin',
                                                   'BMI',
                                                   'DiabetesPedigreeFunction',
                                                   'Age'])])),
                ('model',
                 GridSearchCV(cv=3,
                              estimator=XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                  

In [26]:
print(cv_pipeline['model'].best_score_)
print(cv_pipeline['model'].best_params_)

0.8875976406822893
{'learning_rate': 0.1, 'n_estimators': 50}


In [27]:
model = XGBClassifier(learning_rate= 0.03, n_estimators= 50)

In [28]:
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.03, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [29]:
model.feature_importances_ 

array([0.02718857, 0.05159067, 0.01908977, 0.02037728, 0.5363626 ,
       0.02529847, 0.01563511, 0.0950227 , 0.        , 0.20943487],
      dtype=float32)

In [30]:
# df.Insulin=np.log(1+df.Insulin)

In [31]:
model = XGBClassifier(learning_rate= 0.1, n_estimators= 50)
my_pipeline = Pipeline(steps=[('preprocessor',preprocessor),('model', model)])
my_pipeline.fit(X_train, y_train)
predictions = my_pipeline.predict(X_valid)
accuracy_score(y_valid,predictions)

0.9025974025974026

In [32]:
my_pipeline['model'].feature_importances_ 

array([0.06769945, 0.10537855, 0.04670389, 0.1344941 , 0.40587786,
       0.0635135 , 0.04944277, 0.12688987], dtype=float32)